## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [2]:
data = list(range(1,1001))
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [3]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [4]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [5]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [9]:
# count
rdd.count()


1000

In [10]:
# first
rdd.first()

1

In [11]:
# take
rdd.take(10)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [12]:
# top
rdd.top(10)

[1000, 999, 998, 997, 996, 995, 994, 993, 992, 991]

In [13]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [6]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000) #rand generates values [0,1)
sales_figures

array([9.51966485e-01, 1.88287960e-02, 3.11458724e-01, 3.16998908e+00,
       3.56918577e+00, 2.06628521e+00, 4.34556142e+00, 2.90899935e+00,
       5.50840623e+00, 6.96021341e+00, 8.79484869e+00, 2.81324239e-01,
       7.89006122e+00, 8.13741087e+00, 1.05158289e+01, 1.11707472e+01,
       2.96312000e+00, 1.37341211e+01, 8.09701806e+00, 1.69129196e+01,
       1.12389429e+01, 7.59356832e+00, 1.46052303e+01, 2.23933593e+01,
       2.55434787e+00, 5.62914589e+00, 1.79362457e+01, 1.86478921e+01,
       4.17035414e-01, 1.88575118e+00, 1.65927488e+01, 1.51364254e+01,
       2.57328255e+01, 1.44685552e+01, 2.24415959e+01, 2.22525046e+00,
       3.62885127e+00, 3.39953903e+01, 1.51131685e+01, 2.92989642e+01,
       1.65632407e+01, 6.54878822e+00, 2.93893491e+01, 7.05932503e+00,
       1.24088282e+01, 3.39170179e+01, 4.00110012e+01, 3.96367193e+01,
       3.96720619e+01, 1.27550499e+01, 3.68301141e+01, 8.30967059e+00,
       4.91075587e+01, 3.14310222e+01, 2.42295024e+01, 2.67903583e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [7]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.take(4)
 

[0.9519664851367728,
 0.01882879602046761,
 0.31145872371683025,
 3.1699890786402416]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [8]:
def sales_tax(num):
    return num * 0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [9]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.8758091663258311,
 0.017322492338830202,
 0.28654202581948385,
 2.9163899523490224,
 3.2836509082810372,
 1.9009823963222043,
 3.997916504823852,
 2.6762794028625265,
 5.067733729610353,
 6.403396338144192]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [10]:
discounted = revenue_minus_tax.map(lambda x: x*0.9)

In [11]:
discounted.take(10)

[0.788228249693248,
 0.015590243104947181,
 0.25788782323753545,
 2.62475095711412,
 2.9552858174529337,
 1.710884156689984,
 3.598124854341467,
 2.408651462576274,
 4.560960356649318,
 5.763056704329773]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [12]:
price_items.map(sales_tax).map(lambda x: x*0.90).top(15)

[780.2420708397975,
 774.0304854824116,
 747.7196010448135,
 745.7747101816988,
 729.2600460287862,
 725.1679973804708,
 718.2009434679566,
 714.5848756422563,
 713.6298445524133,
 713.4421840306707,
 708.3346756215849,
 707.8163397278823,
 705.6082537223457,
 705.2820404850462,
 694.8583614177605]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [13]:
discounted.toDebugString()

b'(10) PythonRDD[6] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[1] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [14]:
mapped = price_items.map(lambda x: (x,x*0.92*0.90))
print(mapped.count())
print(mapped.take(10))

1000
[(0.9519664851367728, 0.788228249693248), (0.01882879602046761, 0.015590243104947181), (0.31145872371683025, 0.25788782323753545), (3.1699890786402416, 2.62475095711412), (3.5691857698706926, 2.9552858174529337), (2.0662852133937, 1.710884156689984), (4.345561418286795, 3.598124854341467), (2.9089993509375285, 2.408651462576274), (5.50840622783734, 4.560960356649318), (6.960213411026295, 5.763056704329773)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [15]:
flat_mapped = price_items.flatMap(lambda x: (x,x*0.92*0.90))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.9519664851367728, 0.788228249693248, 0.01882879602046761, 0.015590243104947181, 0.31145872371683025, 0.25788782323753545, 3.1699890786402416, 2.62475095711412, 3.5691857698706926, 2.9552858174529337]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [17]:
# use the filter function
selected_items = discounted.filter(lambda x: x>300)
selected_items.count()

262

In [18]:
selected_items.take(5)

[308.71988102373695,
 301.121628827666,
 318.0843367380809,
 308.497937144826,
 307.09518904165316]

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [19]:
selected_items.reduce(lambda x,y: x + y)

122038.25790736586

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [20]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(32, 308.71988102373695),
 (49, 301.121628827666),
 (12, 318.0843367380809),
 (26, 308.497937144826),
 (44, 307.09518904165316),
 (8, 334.51175472151016),
 (13, 336.17258805428366)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [21]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x, y: x + y)
total_spent.take(5)

[(40, 941.93864846082),
 (50, 2372.187573280348),
 (30, 1810.5253086912612),
 (20, 2660.0071861563783),
 (10, 1181.002145743018)]

In [22]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1], ascending = False).collect()

[(29, 5676.272345562274),
 (37, 4254.107638197173),
 (12, 4250.527945414384),
 (49, 4234.891859543373),
 (16, 4176.132378954669),
 (23, 3887.8074427486354),
 (7, 3774.390996478948),
 (22, 3679.868570970422),
 (46, 3577.9006797911807),
 (4, 3486.4078298771674),
 (32, 3431.013263706772),
 (48, 3351.879590936597),
 (26, 3218.4685644993106),
 (21, 3197.5928016275316),
 (19, 3175.0671841285525),
 (27, 3114.664210187032),
 (25, 2896.219254239123),
 (5, 2885.4019299395923),
 (1, 2823.32261514944),
 (31, 2728.043212534397),
 (47, 2663.8444887265605),
 (20, 2660.0071861563783),
 (8, 2511.067674040384),
 (41, 2380.72116659102),
 (50, 2372.187573280348),
 (43, 2259.9054592593375),
 (39, 2191.4111137940695),
 (24, 2146.2891770726446),
 (3, 2136.8119156934085),
 (11, 2116.3772800208662),
 (9, 2064.7416124083857),
 (2, 2057.1455289382693),
 (17, 2002.350434545659),
 (34, 1839.6985994284876),
 (44, 1833.8765672989614),
 (30, 1810.5253086912612),
 (14, 1698.6602351766483),
 (28, 1464.639516193287),
 (

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [23]:
total_items = sales_data.countByKey()
sorted(total_items.items(), key=lambda kv: kv[1], reverse=True)

[(46, 10),
 (36, 9),
 (26, 8),
 (13, 8),
 (27, 8),
 (24, 8),
 (17, 8),
 (1, 8),
 (15, 8),
 (6, 7),
 (40, 7),
 (21, 7),
 (9, 7),
 (19, 7),
 (18, 7),
 (11, 7),
 (38, 7),
 (12, 6),
 (43, 6),
 (49, 6),
 (39, 5),
 (10, 5),
 (5, 5),
 (31, 5),
 (3, 5),
 (48, 5),
 (2, 5),
 (34, 5),
 (45, 5),
 (33, 4),
 (23, 4),
 (25, 4),
 (41, 4),
 (30, 4),
 (8, 4),
 (37, 4),
 (44, 4),
 (32, 4),
 (7, 4),
 (29, 3),
 (50, 3),
 (14, 3),
 (42, 3),
 (35, 3),
 (28, 3),
 (20, 3),
 (16, 2),
 (4, 2),
 (22, 2),
 (47, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
